In [40]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
#from pandas_profiling import ProfileReport
import warnings 

warnings.filterwarnings('ignore')
%matplotlib inline

# Carregando os dados

In [41]:
training = pd.read_csv('../input/renner_desafio_pucrs_treino.csv')

# Tratamentos necessários
1 - Dropar colunas desnecessárias: 'item', 'semana_comercial' <br>
2 - Removendo linhas com vendas 'GS' (Guide Shop)<br>
3 - Alterar valores negativos de venda para zero (verificar onde os zeros estao distribuidos no dataset)<br>
4 - Alterar variaveis categóricas para variaveis numéricas<br>
5 - Transformar 'data_semana_comercial' para datetime e colocar como índice<br>
--------------------------------------------------------------------------------------<br>
4 - Tratar linhas duplicadas após dropar 'semana_comercial' (Garantir uma linha por item, loja e data_semana_comercial)<br>
7 - Separar entre dataset de treino para loja fisica e web<br>
8 - Dropas colunas desnecessárias para o dataset web<br>

In [47]:
df_train = training.copy()

In [57]:
#pipeline de pré-processamento
def sales_pipeline_v1(dataframe):
    df_pipe = dataframe.copy()
    print('Dataframe carregado!')

    #Dropando colunas desnecessárias
    df_pipe.drop(['item', 'semana_comercial'], axis=1, inplace=True)
    print("Colunas 'item', 'semana_comercial' dropadas!")

    #Dropando linhas com vendas Guide Shop ('GS')
    df_pipe = df_pipe.loc[df_pipe['clima'] != 'GS']
    print('Vendas Guide Shop (GS) dropadas!')

    #Alterando vendas negativas para zero
    df_pipe.loc[df_pipe.venda < 0, 'venda'] = 0
    print('Vendas negativas alteradas para zero!')

    #Transformando variaveis categóricas em variaveis numéricas
    df_pipe['localidade'] = df_pipe['localidade'].map({ 'br':2, 'uy':1, 'ar':0 })
    df_pipe['loja_tamanho'] = df_pipe['loja_tamanho'].map({ 'PP':0, 'P':1, 'M':2, 'G':3, 'GG':4 })
    df_pipe['clima'] = df_pipe['clima'].map({ 'SPO':5, 'RIO':4, 'NOR':3, 'SUL':2 ,'URU':1 ,'ARG': 0, 'W':'W'})
    print('Variaveis categoricas transformadas em numéricas!')

    #Transformando a coluna 'data_semana_comercial' em datetime e index do dataframe
    date = pd.to_datetime(df_train['data_semana_comercial'])
    df_train.index = date
    print("coluna 'data_semana_comercial' transformada em datetime e index do dataframe")

    #Tratando dataset para lojas web
    df_web = df_pipe[df_pipe['categoria_loja'] == 'web']
    del df_web['categoria_loja']
    del df_web['clima']
    del df_web['localidade']
    del df_web['loja_tamanho']
    print('Dataset de treino para lojas web criado')
    

    #Tratando dataset para lojas web
    df_fisica = df_pipe[df_pipe['categoria_loja'] == 'fisica']
    del df_fisica['categoria_loja']
    print('Dataset de treino para lojas fisicas criado')
    
    #Criando csv's com os dados tratados e separados
    df_web.to_csv('../input/clean_data/renner_web_treino.csv')
    df_fisica.to_csv('../input/clean_data/renner_fisica_treino.csv')
    print('CSVs criados e salvos na pasta input/clean_data')
    return df_web, df_fisica


In [58]:
df_web, df_fisica = sales_pipeline_v1(df_train)

Dataframe carregado!
Colunas 'item', 'semana_comercial' dropadas!
Vendas Guide Shop (GS) dropadas!
Vendas negativas alteradas para zero!
Variaveis categoricas transformadas em numéricas!
coluna 'data_semana_comercial' transformada em datetime e index do dataframe
Dataset de treino para lojas web criado
Dataset de treino para lojas fisicas criado
CSVs criados e salvos na pasta input/clean_data


In [59]:
df_web.head()

,cod_loja,data_semana_comercial,venda
data_semana_comercial,,,
2022-06-05,407,2022-06-05,188
2021-05-23,407,2021-05-23,136
2021-05-30,406,2021-05-30,926
2022-04-03,407,2022-04-03,152
2022-07-24,407,2022-07-24,33


In [60]:
df_fisica.head()

,cod_loja,data_semana_comercial,venda,localidade,loja_tamanho,clima
data_semana_comercial,,,,,,
2021-07-25,58,2021-07-25,51,2,3,4
2022-05-22,63,2022-05-22,86,2,3,4
2021-08-01,84,2021-08-01,62,2,3,4
2022-05-29,2,2022-05-29,171,2,3,2
2021-08-08,35,2021-08-08,13,2,3,5


### Próximos passos:
* verificar a distribuição dos zeros(das vendas negativas) no novo dataset, para não criar vies na previsão de alguma regiao, loja, etc 
* verificar se o groupby abaixo está funcionando
* talvez testar mudar a escala das features caso os modelos nao funcionem bem


In [76]:
df_agrupado = df.groupby('data_semana_comercial').agg(
                                                    {'venda': 'sum', 
                                                    'cod_loja': 'first', 
                                                    'categoria_loja': 'first', 
                                                    'localidade': 'first', 
                                                    'loja_tamanho': 'first', 
                                                    'clima': 'first'}).reset_index()

In [78]:
df_agrupado.head()

,data_semana_comercial,venda,cod_loja,categoria_loja,localidade,loja_tamanho,clima
0,2017-01-01,384,47,fisica,br,M,SPO
1,2017-01-08,344,150,fisica,br,G,RIO
2,2017-01-15,289,132,fisica,br,P,SUL
3,2017-01-22,305,43,fisica,br,M,SPO
4,2017-01-29,848,67,fisica,br,M,SUL


In [81]:
df_agrupado.shape

(295, 7)